In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
file_list = [f for f in os.listdir('Data_Raw') if f.endswith('.txt')]

In [3]:
delimiter = ','  # Change this to the actual delimiter if it's different
data_all = pd.DataFrame()
for file_path in file_list:
    # Read the text file into a Pandas DataFrame
    df = pd.read_csv('Data_Raw/' + file_path, delimiter=delimiter, header=None)
    df = df.iloc[:, :12]
    names = ['TimeStamp', 'Station', 'District', 'Freeway_#', 'Dir_Travel', 'Type', 'Length', 'Samples',
             'Observed_%', 'Total_Flow', 'Occupancy', 'Speed']
    df.columns = names
    df = df[df['Type']=="ML"]

    data_all = pd.concat([data_all, df], ignore_index=True)

In [4]:
data_all_observed = data_all[data_all['Observed_%'] > 0]
summary_observed = data_all_observed.groupby('Station').size().reset_index(name='Count')

In [5]:
metadata = pd.read_csv('Data_Raw/Archive/d07_text_meta_2022_07_15.txt', delimiter='\t')
metadata = metadata[metadata['Type']=="ML"]

In [6]:
metadata_obs = pd.merge(metadata, summary_observed, left_on='ID', right_on='Station' , how='left')

In [7]:
#metadata_obs.to_csv('Data_Raw/Station_MetaData.csv', index=False)

## Station Selection

In [8]:
station_selected = pd.read_csv('Data_Raw/CA60_Exported.csv')

In [9]:
station_selected['PM_Dir'] = station_selected['Abs_PM'].max()-station_selected['Abs_PM']

In [10]:
station_selected['Station_Number'] = station_selected['Abs_PM'].rank(ascending=False)
station_selected['Station_PM'] = station_selected['PM_Dir'].round(1) * 10

ID_to_remove = [774125, 773246, 717282]
station_selected = station_selected[~station_selected['ID'].isin(ID_to_remove)]

In [11]:
order_match = station_selected[['ID', 'Station_PM']]

In [12]:
ID_selected = station_selected['ID']
data_all_select = data_all[data_all['Station'].isin(ID_selected)]
data_all_select['TimeStamp'] = pd.to_datetime(data_all_select['TimeStamp'])
data_all_select['TimeDiff'] = ((data_all_select['TimeStamp'] - data_all_select['TimeStamp'].min()).dt.total_seconds())/60
data_all_select['TimeStep'] = 1 + (data_all_select['TimeDiff'] // 5)

/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_54429/440614197.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all_select['TimeStamp'] = pd.to_datetime(data_all_select['TimeStamp'])
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_54429/440614197.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all_select['TimeDiff'] = ((data_all_select['TimeStamp'] - data_all_select['TimeStamp'].min()).dt.total_seconds())/60
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_5

In [13]:
#station_selected[station_selected['PM_Dir_round'].duplicated(keep=False)]

In [14]:
data_all_select = pd.merge(data_all_select, order_match, left_on='Station', right_on='ID' , how='left')

In [15]:
Flow = data_all_select[['TimeStep', 'Station_PM', 'Total_Flow']]
Flow['TimeStep'] = Flow['TimeStep'].astype(int)
Flow['Station_PM'] = Flow['Station_PM'].astype(int)

Occupancy = data_all_select[['TimeStep', 'Station_PM', 'Occupancy']]
Occupancy['TimeStep'] = Occupancy['TimeStep'].astype(int)
Occupancy['Station_PM'] = Occupancy['Station_PM'].astype(int)

Speed = data_all_select[['TimeStep', 'Station_PM', 'Speed']]
Speed['TimeStep'] = Speed['TimeStep'].astype(int)
Speed['Station_PM'] = Speed['Station_PM'].astype(int)

/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_54429/3535391722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Flow['TimeStep'] = Flow['TimeStep'].astype(int)
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_54429/3535391722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Flow['Station_PM'] = Flow['Station_PM'].astype(int)
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_54429/3535391722.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

## Final Data Cleaning

In [16]:
"""
Create a table with timestep and PM (posted Miles) as one-one match
"""
timestep_value = [i for i in range(1, Occupancy['TimeStep'].max() + 1) for _ in range(Occupancy['Station_PM'].max() + 1)]
PM_values = list(range(Occupancy['Station_PM'].max() + 1)) * Occupancy['TimeStep'].max()
Time_PM = pd.DataFrame({'TimeStep': timestep_value, 'Station_PM': PM_values})

In [17]:
Occupancy_all = pd.merge(Time_PM, Occupancy, on=['TimeStep', 'Station_PM'] , how='left')
Flow_all = pd.merge(Time_PM, Flow, on=['TimeStep', 'Station_PM'] , how='left')
Speed_all = pd.merge(Time_PM, Speed, on=['TimeStep', 'Station_PM'] , how='left')

In [19]:
Occupancy_all_wide = Occupancy_all.pivot(index='Station_PM', columns='TimeStep', values='Occupancy')
Flow_all_wide = Flow_all.pivot(index='Station_PM', columns='TimeStep', values='Total_Flow')
Speed_all_wide = Speed_all.pivot(index='Station_PM', columns='TimeStep', values='Speed')

In [20]:
Flow_wide = Flow.pivot(index='Station_PM', columns='TimeStep', values='Total_Flow')
Occupancy_wide = Occupancy.pivot(index='Station_PM', columns='TimeStep', values='Occupancy')
Speed_wide = Speed.pivot(index='Station_PM', columns='TimeStep', values='Speed')

In [21]:
Flow_all_wide.to_csv('Data_Clean/Flow_all.csv')
Occupancy_all_wide.to_csv('Data_Clean/Occupancy_all.csv')
Speed_all_wide.to_csv('Data_Clean/Speed_all.csv')

In [22]:
Flow_wide.to_csv('Data_Clean/Flow.csv')
Occupancy_wide.to_csv('Data_Clean/Occupancy.csv')
Speed_wide.to_csv('Data_Clean/Speed.csv')

In [24]:
Flow.to_csv('Data_Clean/Flow_Long.csv', index=False)
Occupancy.to_csv('Data_Clean/Occupancy_Long.csv', index=False)
Speed.to_csv('Data_Clean/Speed_Long.csv', index=False)

In [25]:
Flow_all.to_csv('Data_Clean/Flow_all_Long.csv', index=False)
Occupancy_all.to_csv('Data_Clean/Occupancy_all_Long.csv', index=False)
Speed_all.to_csv('Data_Clean/Speed_all_Long.csv', index=False)